In [516]:
import nltk
from nltk.util import ngrams
from nltk.collocations import *
from nltk.metrics.association import QuadgramAssocMeasures
from nltk import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from collections import Counter
from string import punctuation
from itertools import chain
import requests
import re

# Endereço da API dos discursos
BABEL_API_URL = 'https://babel.labhackercd.leg.br/api/v1/manifestations?manifestation_type__id=1'

# Pegando os 200 primeiros discursos da API do Babel
speeches = []
data = requests.get(BABEL_API_URL).json()['results']
for i in range(2, 100):
    data += requests.get(BABEL_API_URL + '&page=%s' % i).json()['results']
    
for speech in data:
    for attr in speech['attrs']:
        if attr['field'] == 'original':
            speeches.append(attr['value'])
            break

In [549]:
# Criando a lista de stopwords em português
EXTRA_STOPWORDS = [
    'sr.', 'nesse', 'deputados', '``', "''", 'empresa', 'trabalhadores', 'tema', 'brasil', 'brasileiro',
    'brasileira', 'sociedade', 'grandes', 'meios', 'principal', 'deputada', 'nesta', 'valor', 'reais',
    'representante', 'brasileiros', 'necessidade', 'quero', 'ser', 'geral', 'todo', 'toda', 'estar',
    'ter', 'parlamentares', 'região', 'forma', 'parte', 'disso', 'debate', 'entregar', 'nessa', 'nome',
    'vez', 'quer', 'primeira', 'soberania', 'justamente', 'ponto', 'presentes', 'faz', 'futuro', 'podem',
    'maneira', 'falar', 'interesses', 'caso', 'espaço', 'entrega', 'deste', 'mesma', 'negócio', 'existe',
    'avançar', 'ano', 'diz', 'próprios', 'criação', 'própria', 'dizendo', 'trazer', 'preocupação', 'ali',
    'ficar', 'desse', 'importância', 'senhor', 'queremos', 'poderia', 'ir', 'próprio', 'área', 'segundo',
    'acontece', 'sei', 'sabem', 'comum', 'mim', 'tratar', 'obrigação', 'falando', 'discurso', 'muitas',
    'acabou', 'falou', 'outro', 'capacidade', 'força','querem', 'significa', 'serviço', 'dados', 'tentar',
    'ninguém', 'gente', 'ideia', 'deputadas', 'dentro', 'fazendo', 'mão', 'época', 'uso', 'fiz', 'último',
    'nenhum', 'números', 'alguma', 'acreditamos', 'achamos', 'pagar', 'paga', 'conjunto', 'contentam',
    'lacunas', 'esperamos', 'digo', 'sente', 'logo', 'pesquisar', 'principais', 'mudar', 'sequer', 'pagou',
    'pagando', 'traz', 'desafio', 'concreto', 'atende', 'tendo', 'aberto', 'curto', 'recebe', 'receber',
    'décadas', 'minutos', 'horas', 'segundos', 'motivo', 'posso', 'dando', 'entra', 'volto', 'construir',
    'algumas', 'passar', 'muita', 'nisso', 'deveria', 'dá', 'construir', 'muito', 'longo', 'muitas', 'outras',
    'existem', 'conseguimos', 'precisamos', 'feita', 'mencionou', 'falei', 'cujo', 'hora', 'dizia', 'venha',
    'conseguindo', 'conseguir', 'votar', 'alguém', 'somente', 'todas', 'fizemos', 'citar', 'saiba', 'boa',
    'deveriam', 'acontecendo', 'algo', 'apresentei', 'sabemos', 'outra', 'junto', 'daqui', 'aconteceu',
    'haver', 'sinto', 'preciso', 'muitos', 'minimamente', 'amanhã', 'ontem', 'partir', 'objetivo', 'opinião',
    'vivemos', 'discutindo', 'agradecemos', 'utilizados', 'contra', 'dona', 'falta', 'possa', 'manhã', 'novas',
    'após', 'pessoas', 'fundamental', 'desses', 'devido', 'item', 'século', 'domingo', 'sábado',
    'presidente', ',', '.', '...', 'é', 'questão', 'art', 'ordem', 'v.exa', ':', 'governo', 'sr', 'agência',
    'aqui', 'vai', 'artigo', '§', 'neste', 'vamos', 'agora', "''", 'fazer', 'mesa', 'ainda', 'porque', 'trata',
    'estrutura', 'sobre', 'então', 'todos', 'obstrução', 'votação', 'presença', 'deputados', 'vou', 'brasil',
    'discutir', 'vigência', 'colocar', 'regimento', 'momento', ';', 'dois', 'dessa', 'medida', 'proposta',
    'casa', 'matéria', 'queria', 'assim', 'possamos', 'microfone', 'certeza', 'hoje', 'profissional', 'deixar',
    'provisória', 'ora', 'base', 'importante', 'veto', 'fala', '!', 'aumento', 'inciso', 'sra.', 'talvez',
    'cima', 'servir', 'nunca', 'dias', 'deus', 'dei', 'entendemos', 'chega', 'possam', 'entendo', 'poderá',
    'celeridade', 'tirar', 'mista', 'fechou-se', 'lado', 'lido', 'repassado', 'demais', 'venho', 'marcar',
    'xiii', 'diálogo', 'podemos', 'apenas', 'poder', 'efeitos', 'pode', 'acordo', 'solicitação',
    'reflexão', '?', 'ausência', 'aprovada', 'lideranças', 'dizer', 'portanto', 'peço', 'recolher',
    'prática', 'pois', 'milhões', 'bilhões', 'melhoria', 'atividade', 'claro', 'saber', 'dar',
    'avanço', 'condições', 'desastre', 'especialmente', 'exatamente', 'política', 'vezes', 'fazê-lo', 'têm',
    'derrubar', 'precisa', 'custo', 'necessária', 'cláusula', 'proposição', '-', 'palavra', 'tempo', 'segundos',
    'fez', 'necessário', 'zero', 'interesse', 'srs', 'sr', 'sras', 'sra', 'deputado', 'presidente', 'é', 'nº',
    's.a.', 'v.exa.', 'v.exa', '#', 'anos', 'º', 'exa', 'mesa', 'legislatura', 'sessão', 'maioria', 'seguinte',
    'mandato', 'bilhões', 'quilômetros', 'ª', 'parabéns', 'membros', 'convido', 'usual', 'biênio',
    'brasil', 'palavra', 'discussão', 'período', 'início', 'pronunciamento', 'suplente', 'atividade', 'ação',
    'ações', 'daqueles', 'diferenças', 'pasta', 'milhares', 'srªs', 'emenda', 'àqueles', 'tamanha', 'mês',
    'capaz', 'km', 'modelo', 'tarefas', 'colegas', 'programa', 'voz', 'pronunciamento',
    'casa', 'sessão', 'deliberativa', 'solene', 'ordinária', 'extraordinária', 'encaminhado', 'orador',
    'divulgar', 'deputado', 'parlamentar', 'projeto', 'proposta', 'requerimento', 'destaque',
    'veto', 'câmara', 'senado', 'congresso', 'país', 'estado', 'brasil', 'lei', 'novo', 'nova',
    'política', 'povo', 'voto', 'partido', 'liderança', 'bancada', 'bloco', 'líder', 'lider', 'frente',
    'governo', 'oposição', 'presença', 'presente', 'passado', 'ausência', 'ausencia', 'ausente', 'obstrução',
    'registrar', 'aprovar', 'rejeitar', 'rejeição', 'sabe', 'matéria', 'materia', 'questão', 'ordem', 'emenda',
    'sistema', 'processo', 'legislativo', 'plenário', 'pedir', 'peço', 'comissão', 'especial', 'permanente',
    'apresentar', 'encaminhar', 'encaminho', 'orientar', 'liberar', 'apoiar', 'situação', 'fato', 'revisão',
    'tempo', 'pauta', 'discutir', 'discussão', 'debater', 'retirar', 'atender', 'colegas', 'autor', 'texto',
    'medida', 'união', 'república', 'audiência', 'audiencia', 'público', 'publico', 'reunião', 'agradecer',
    'solicitar', 'assistir', 'contrário', 'favorável', 'pessoa', 'comemorar', 'ato', 'momento', 'diretora',
    'possível', 'atenção', 'agradeço', 'naquele', 'necessárias', 'presidenta', 'compromisso', 'geradas',
    'primeiro', 'simplesmente', 'ideal', 'argumento', 'i', 'válido', 'envolvidos', 'nesse', 'aspecto',
    'existentes', 'normativo', 'irá', 'nada', 'melhor', 'esperarmos', 'pouco', 'resolvermos', 'problema',
    'postura', 'faltas', 'declara', '%', 'dia', 'obrigado', 'agradeço', 'agradecido', 'população',
    'maior', 'cada', 'bem', 'mundo', 'desta', 'mil', 'sendo', 'outros', '$', '!', '@', '#', '&', '(', ')',
    'r', 'sempre', 'além', 'semana', 'relação', 'onde', 'meio', 'inclusive', 'lá', 'vem', 'menos', 'menor',
    'qualquer', 'desde', 'ontem', 'hoje', 'exemplos', 'exemplo', 'tão', 'fim', 'janeiro', 'fevereiro', 'março',
    'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'alguns',
    'durante', 'gostaria', 'três', 'conta', 'feito', 'através', 'antes', 'depois', 'verdade', 'bom', 'quase',
    'setor', 'aí', 'disse', 'principalmente', 'final', 'vão', 'coisa', 'ver', 'sentido', 'vários',
    'nenhuma', 'quanto', 'infelizmente', 'felizmente', 'número', 'duas', 'dois', 'tanto', 'acho', 'achar',
    'enquanto', 'deve', 'apelo', 'papel', 'últimos', 'faço', 'fazer', 'garantir', 'garantia', 'fica',
    'obrigado..', 'assunto', 'sido', 'vir', 'incrementar', 'central', 'aproximado', 'aproximadamente',
    'hipotética', 'hipotese', 'hipótese', 'superiores', 'entende', 'pedido', 'oradora', 'tal', 'v.exas',
    'favor', 'vota', 'nº', 'srª', 'vista', 'sim', 'dito', 'tudo', 'obrigado', 'º', 'profundamente', 'custódio',
    'divulgado', 'características', 'perfeito', 'começarmos', 'nomes', 'amigo', 'possibilidade', 'mensagem',
    'come', 'parabenizar', 'começar', 'hs', 'atendimento', 'povos', '¯', 'ocorreu', 'entanto', 'diante',
    'defender', 'dr.', '“', '”', '•', 'v.', './', 'és', 'senhoras', 'senhores', 'tipo', 'várias', 'gerais',
    'quais', 'dessas', 'deu', 'havia', 'devem', 'enfim', 'apesar', 'passa', 'chegou', 'vêm', 'parece', 'u'
]

ONEGRAM_STOPWORDS = [
    'grande', 'nacional', 'são', 'e', 'de', 'das', 'dos', 'da', 'do', 'federal', 'cedo', 'urgência', 'equipe',
    'produtos', 'serviços', 'pequeno', 'total', 'podermos', 'consenso', 'popular', 'mérito', 'único', 'pública',
    'escolha', 'acesso', 'pilotos', 'trabalhar', 'ministério', 'países', 'combate', 'estados', 'vida', 'cidade',
    'municípios', 'histórico', 'defesa', 'município', 'prefeito', 'ii', 'santa', 'vereadora', 'centro',
    'prefeitura', 'governador', 'código', 'apoio', 'exercício', 'categoria', 'campo', 'kit',
    'ministro', 'social', 'recursos', 'direito', 'empresas', 'comunicação', 'democracia',
    'tribuna', 'história', 'respeito', 'luta', 'oportunidade', 'dinheiro', 'públicos',
    'civil', 'qualidade', 'políticas', 'sociais', 'registro', 'públicas', 'crescimento',
    'responsabilidade', 'participação', 'importantes', 'gestão', 'minas', 'cidades', 'lugar',
    'problemas', 'decisão', 'mulher', 'nobre', 'capital', 'aprovação', 'humanos', 'estadual',
    'internacional', 'senador', 'redução', 'realmente', 'realidade', 'plano', 'partidos',
    'conselho', 'posição', 'medidas', 'termos', 'divulgação', 'econômico', 'federais', 'fiscal',
    'emprego', 'maiores', 'rede', 'ruas', 'regional', 'continuar', 'profissionais', 'sob',
    'homens', 'político', 'atual', 'nação', 'meses', 'grupo', 'áreas', 'fundo', 'iniciativa',
    'executivo', 'cerca', 'cidadão', 'prazo', 'homem', 'trabalhador', 'órgãos', 'campanha',
    'controle', 'mínimo', 'mundial', 'dúvida', 'legislação', 'relatório', 'emendas', 'atividades',
    'razão', 'resultado', 'instituições', 'brasileiras', 'líderes', 'última', 'secretário', 'precisam',
    'criar', 'movimento', 'data', 'fazem', 'novos', 'casos', 'ambiente', 'administração', 'distrito',
    'pesquisa', 'relator', 'cumprimento', 'causa', 'informações', 'evento', 'aliás', 'superior',
    'filho', 'tarde', 'caminho', 'dificuldades', 'risco', 'publicação', 'sobretudo', 'coisas',
    'obrigada', 'santo', 'solicito', 'cargos', 'condição', 'próximo', 'secretaria', 'formação',
    'penal', 'forte', 'representa', 'aprovado', 'acima', 'políticos', 'setores', 'chegar',
    'espírito', 'prefeitos', 'grave', 'solução', 'governos', 'conhecimento', 'espero',
    'preço', 'mudança', 'instituto', 'tributária', 'amigos', 'levar', 'diversos',
    'municipal', 'dado', 'filhos', 'proteção', 'pagamento', 'federação', 'entidades',
    'br-', 'funcionários', 'média', 'organização', 'veículos', 'difícil', 'patrimônio',
    'marco', 'geração', 'diversas', 'honra', 'aumentar', 'movimentos', 'idade', 'passou',
    'inclusão', 'responsável', 'única', 'busca', 'questões', 'operação', 'participar',
    'pequenos', 'ajudar', 'regime', 'vítimas', 'pior', 'orgulho', 'unidos', 'embora',
    'forças', 'inteiro', 'modo', 'simples', 'programas', 'legislativa', 'caixa', 'leis',
    'passada', 'cidadãos', 'dignidade', 'associação', 'absolutamente', 'contribuição',
    'trata-se', 'esforço', 'representantes', 'fizeram', 'vereador', 'ficou', 'volta',
    'quadro', 'lembrar', 'concluir', 'votos', 'classe', 'atuação', 'médio', 'receita',
    'palavras', 'nível', 'encontro', 'milhão', 'diferente', 'local', 'estaduais',
    'constitucional', 'recurso', 'certamente', 'nossa'
]


# Método para limpar texto do discurso tirando as notas do taquigrafo
def clear_speech(text):
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'[OA] SRA?[\w\s.]+-', '', text)
    text = re.sub(r'PRONUNCIAMENTO[\sA-Z]+\s', '', text)
#     text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s[\.\"]+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[Vv]\.[Ee][Xx][Aa]\.', 'v.exa', text)
    text = re.sub(r'[Aa][Rr][Tt]\.', 'art', text)
    text = re.sub(r'[Ss][Rr][Ss]?\.', 'sr', text)
    text = re.sub(r'[Ss][Rr][Aa][Ss]?\.', 'sr', text)
    text = re.sub(r'\d', '', text)
    return text.strip()


def get_tokens(speeches, extra_stopwords=None):
    """
    Função que retorna tokens de discursos removendo as "stopwords".
    Argumentos:
        speeches: Recebe uma lista de discursos.
        stopwords: Recebe uma lista de palavras a serem retiradas dos textos.
    Retorna:
        Uma lista palavras do discurso que não estão nas "stopwords".
    """
    special_stopwords = ['são', 'nossa']
    stopwords = nltk_stopwords.words('portuguese') + list(punctuation) + EXTRA_STOPWORDS
    stopwords = [word for word in stopwords if word not in special_stopwords]
    if extra_stopwords:
        stopwords += extra_stopwords
    tokens = []
    for text in speeches[:200]:
        text = clear_speech(text)
        tokens += [i for i in word_tokenize(text.lower(), language='portuguese') if i not in stopwords]
    
    return tokens


def clean_bigrams(bigrams):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
#     tokens = get_tokens(speeches)
    tokens = []
    for text in speeches: # Gera bigrams com todos os discursos da base
        text = clear_speech(text)
        tokens += [i for i in word_tokenize(text.lower(), language='portuguese')]
    bigram_f = BigramCollocationFinder.from_words(tokens)
    bigram_f.apply_freq_filter(10) # filtra bigrams com frequência maior que 10
    bigram_score_pmi = bigram_f.score_ngrams(bigram_measures.pmi)
    bigram_list = [x[0] for x in bigram_score_pmi if x[1] >= 8.0] # Lista de bigrams com PMI > 8.0
    bigram_result = [bigram for bigram in bigrams if bigram[0] in bigram_list]

    return bigram_result


def ngrams_by_limit(tokens, n, limit):
    """
    Função que retorna a quantidade de amostras do gênero masculino e feminino.
    Argumentos:
        tokens: Recebe uma lista de tokens já processados pelo nltk.word_tokenize.
        n: Recebe o número de palavras que deseja dividir o ngram.
        limit: Recebe o limite mínimo de ocorrência.
    Retorna:
        Uma lista de ngrams com ocorrência maior que "limite" e com "n" palavras.
    """
    ngrams_count = Counter(ngrams(tokens, n)).most_common()
    result = [x for x in ngrams_count if x[1] >= limit]
    return result

In [550]:
#Definimos os tokens
tokens = get_tokens(speeches) # tokens de 200 discursos

# Determinamos o limite de ocorrências usado no algoritmo
limit = Counter(tokens).most_common(int(len(speeches) * 0.2))[-1][1]
if limit < 3:
    limit = 3

print("Limite: %s" % limit)

Limite: 9


In [551]:
bigrams = ngrams_by_limit(tokens, 2, limit)
bigrams # Bigrams de maior ocorrência

[(('rio', 'grande'), 57),
 (('segurança', 'pública'), 52),
 (('grande', 'sul'), 42),
 (('supremo', 'tribunal'), 31),
 (('tribunal', 'federal'), 30),
 (('agentes', 'comunitários'), 28),
 (('porto', 'alegre'), 26),
 (('michel', 'temer'), 22),
 (('são', 'paulo'), 22),
 (('comunitários', 'saúde'), 22),
 (('presunção', 'inocência'), 19),
 (('técnicos', 'industriais'), 16),
 (('exercício', 'ilegal'), 15),
 (('saúde', 'educação'), 14),
 (('habeas', 'corpus'), 14),
 (('santa', 'catarina'), 13),
 (('caravana', 'lula'), 13),
 (('carlos', 'manato'), 13),
 (('rodrigo', 'maia'), 12),
 (('grande', 'norte'), 11),
 (('técnicos', 'agrícolas'), 11),
 (('marielle', 'franco'), 11),
 (('polícia', 'civil'), 11),
 (('ministro', 'saúde'), 10),
 (('segunda', 'instância'), 10),
 (('são', 'francisco'), 9),
 (('taxa', 'juros'), 9),
 (('conselho', 'federal'), 9),
 (('caixa', 'econômica'), 9),
 (('econômica', 'federal'), 9),
 (('mato', 'grosso'), 9),
 (('combate', 'endemias'), 9),
 (('nossa', 'senhora'), 9),
 (('ex

In [552]:
new_bigrams = clean_bigrams(bigrams)
new_bigrams # Bigrams filtrados usando o PMI em 2000 discursos

[(('segurança', 'pública'), 52),
 (('supremo', 'tribunal'), 31),
 (('tribunal', 'federal'), 30),
 (('agentes', 'comunitários'), 28),
 (('porto', 'alegre'), 26),
 (('michel', 'temer'), 22),
 (('são', 'paulo'), 22),
 (('técnicos', 'industriais'), 16),
 (('exercício', 'ilegal'), 15),
 (('habeas', 'corpus'), 14),
 (('santa', 'catarina'), 13),
 (('carlos', 'manato'), 13),
 (('rodrigo', 'maia'), 12),
 (('técnicos', 'agrícolas'), 11),
 (('marielle', 'franco'), 11),
 (('polícia', 'civil'), 11),
 (('segunda', 'instância'), 10),
 (('caixa', 'econômica'), 9),
 (('mato', 'grosso'), 9),
 (('nossa', 'senhora'), 9),
 (('ex-presidente', 'lula'), 9),
 (('código', 'penal'), 9),
 (('agricultura', 'familiar'), 9),
 (('medicina', 'veterinária'), 9)]

In [553]:
def clean_tokens(tokens, bigrams=[], trigrams=[], extra_stopwords=None):   
    if bigrams:
        pos_bigram = []
        for i in range(len(tokens)-1):
            for x, y in bigrams:
                if tokens[i] == x and tokens[i+1] == y:
                    pos_bigram.append(i)

        for pos in reversed(pos_bigram):
            del tokens[pos:pos+2]
    
    if trigrams:
        pos_trigram = []
        for i in range(len(tokens)-2):
            for x, y, z in trigrams:
                if tokens[i] == x and tokens[i+1] == y and tokens[i+2] == z:
                    pos_trigram.append(i)

        for pos in reversed(pos_trigram):
            del tokens[pos:pos+3]
    
    if extra_stopwords:
        new_tokens = [token for token in tokens if token not in extra_stopwords]
    else:
        new_tokens = tokens

    return new_tokens

In [554]:
stop_bigrams = []

if new_bigrams:
    stop_bigrams = list(list(zip(*new_bigrams))[0])

trigram_tokens = clean_tokens(tokens, stop_bigrams)
trigrams = ngrams_by_limit(trigram_tokens, 3, limit)
trigrams

[(('rio', 'grande', 'sul'), 42), (('rio', 'grande', 'norte'), 11)]

In [555]:
stop_bigrams = stop_trigrams = []

if new_bigrams:
    stop_bigrams = list(list(zip(*new_bigrams))[0])
if trigrams:
    stop_trigrams = list(list(zip(*trigrams))[0])
    
onegram_tokens = clean_tokens(tokens, stop_bigrams, stop_trigrams, ONEGRAM_STOPWORDS)
onegrams = ngrams_by_limit(onegram_tokens, 1, limit)
onegrams

[(('saúde',), 120),
 (('violência',), 88),
 (('lula',), 88),
 (('rio',), 88),
 (('trabalho',), 79),
 (('educação',), 77),
 (('constituição',), 54),
 (('crime',), 46),
 (('caravana',), 41),
 (('segurança',), 39),
 (('polícia',), 38),
 (('obras',), 33),
 (('justiça',), 33),
 (('marielle',), 33),
 (('policiais',), 31),
 (('sergipe',), 31),
 (('ódio',), 30),
 (('obra',), 29),
 (('jovens',), 28),
 (('intervenção',), 27),
 (('agentes',), 26),
 (('morte',), 26),
 (('militar',), 26),
 (('famílias',), 25),
 (('prisão',), 25),
 (('hildo',), 24),
 (('dilma',), 24),
 (('temer',), 23),
 (('água',), 23),
 (('ceará',), 23),
 (('sul',), 23),
 (('vetos',), 23),
 (('tese',), 23),
 (('pena',), 23),
 (('josé',), 22),
 (('corrupção',), 22),
 (('investimentos',), 22),
 (('economia',), 22),
 (('sindicato',), 22),
 (('reforma',), 21),
 (('projetos',), 21),
 (('cultura',), 21),
 (('golpe',), 21),
 (('supremo',), 21),
 (('taxa',), 20),
 (('juros',), 20),
 (('medicina',), 20),
 (('inocência',), 20),
 (('pobres',

In [556]:
result_tokens = onegrams + new_bigrams + trigrams
result_tokens.sort(key=lambda x: x[1], reverse=True)
result_tokens

[(('saúde',), 120),
 (('violência',), 88),
 (('lula',), 88),
 (('rio',), 88),
 (('trabalho',), 79),
 (('educação',), 77),
 (('constituição',), 54),
 (('segurança', 'pública'), 52),
 (('crime',), 46),
 (('rio', 'grande', 'sul'), 42),
 (('caravana',), 41),
 (('segurança',), 39),
 (('polícia',), 38),
 (('obras',), 33),
 (('justiça',), 33),
 (('marielle',), 33),
 (('policiais',), 31),
 (('sergipe',), 31),
 (('supremo', 'tribunal'), 31),
 (('ódio',), 30),
 (('tribunal', 'federal'), 30),
 (('obra',), 29),
 (('jovens',), 28),
 (('agentes', 'comunitários'), 28),
 (('intervenção',), 27),
 (('agentes',), 26),
 (('morte',), 26),
 (('militar',), 26),
 (('porto', 'alegre'), 26),
 (('famílias',), 25),
 (('prisão',), 25),
 (('hildo',), 24),
 (('dilma',), 24),
 (('temer',), 23),
 (('água',), 23),
 (('ceará',), 23),
 (('sul',), 23),
 (('vetos',), 23),
 (('tese',), 23),
 (('pena',), 23),
 (('josé',), 22),
 (('corrupção',), 22),
 (('investimentos',), 22),
 (('economia',), 22),
 (('sindicato',), 22),
 (('